In [1]:
!pip install tensorflow --upgrade 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.3/601.3 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 13.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 22.9 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 23.5.26
    Uninstalling flatbuffers-23.5.26:
      Successfully uninstalled flatbuffers-23.5.26
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.1
    Uninstalling tensorboard-2.15.1:
      Successfully uninstalled tensorboard-2.15.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.15.0
    Uninstalling tensorflow-2.15.0:
      Successfully uninstalled tensorflow-2.15.0
ERROR: pip's dependency resolver does not currentl

In [70]:
#Required Libraries
import numpy as np
import pandas as pd
import os
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow import *
from tensorflow import keras
from keras.models import Sequential, Model
from tensorflow.keras.applications import *
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout,BatchNormalization,GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score
import warnings
warnings.filterwarnings("ignore")

In [5]:
#Copy and paste the training and testing folder in training and testing variable
training_folder=r"/kaggle/input/leaf-disease-detection-dataset/dataset/train"
testing_folder=r"/kaggle/input/leaf-disease-detection-dataset/dataset/test"

In [6]:
# Find the Number of classess
number_of_classes=len(os.listdir(training_folder))
print(f"Number of classes: {number_of_classes}")

Number of classes: 38


I0000 00:00:1722691857.182895      34 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1722691857.327121      34 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1722691857.327379      34 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1722691857.330096      34 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [4]:
#Find the Number of images in Training folder
total_number_of_images_in_training=0
folder_path=os.listdir(training_folder)
for i,folder in enumerate(folder_path):
    number_of_images=len(os.listdir(training_folder+'/'+folder))
    total_number_of_images_in_training+=number_of_images
    print(f"{i} , In {folder} folder contains {number_of_images} images.")

print(f"Total Number of Images in Training Folder: {total_number_of_images_in_training}")


0 , In Tomato___Late_blight folder contains 1851 images.
1 , In Tomato___healthy folder contains 1926 images.
2 , In Grape___healthy folder contains 1692 images.
3 , In Orange___Haunglongbing_(Citrus_greening) folder contains 2010 images.
4 , In Soybean___healthy folder contains 2022 images.
5 , In Squash___Powdery_mildew folder contains 1736 images.
6 , In Potato___healthy folder contains 1824 images.
7 , In Corn_(maize)___Northern_Leaf_Blight folder contains 1908 images.
8 , In Tomato___Early_blight folder contains 1920 images.
9 , In Tomato___Septoria_leaf_spot folder contains 1745 images.
10 , In Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot folder contains 1642 images.
11 , In Strawberry___Leaf_scorch folder contains 1774 images.
12 , In Peach___healthy folder contains 1728 images.
13 , In Apple___Apple_scab folder contains 2016 images.
14 , In Tomato___Tomato_Yellow_Leaf_Curl_Virus folder contains 1961 images.
15 , In Tomato___Bacterial_spot folder contains 1702 images.
16 ,

In [5]:
#Find the Number of images in Testing folder
total_number_of_images_in_testing=0
folder_path=os.listdir(testing_folder)
for i,folder in enumerate(folder_path):
    number_of_images=len(os.listdir(testing_folder+'/'+folder))
    total_number_of_images_in_testing+=number_of_images
    print(f"{i}, In {folder} folder contains {number_of_images} images.")

print(f"Total Number of Images in Testing Folder: {total_number_of_images_in_testing}")

0, In Tomato___Late_blight folder contains 463 images.
1, In Tomato___healthy folder contains 481 images.
2, In Grape___healthy folder contains 423 images.
3, In Orange___Haunglongbing_(Citrus_greening) folder contains 503 images.
4, In Soybean___healthy folder contains 505 images.
5, In Squash___Powdery_mildew folder contains 434 images.
6, In Potato___healthy folder contains 456 images.
7, In Corn_(maize)___Northern_Leaf_Blight folder contains 477 images.
8, In Tomato___Early_blight folder contains 480 images.
9, In Tomato___Septoria_leaf_spot folder contains 436 images.
10, In Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot folder contains 410 images.
11, In Strawberry___Leaf_scorch folder contains 444 images.
12, In Peach___healthy folder contains 432 images.
13, In Apple___Apple_scab folder contains 504 images.
14, In Tomato___Tomato_Yellow_Leaf_Curl_Virus folder contains 490 images.
15, In Tomato___Bacterial_spot folder contains 425 images.
16, In Apple___Black_rot folder cont

In [6]:
#To Get the filename with pathname and folder name for label for Training Folder
file_path=[] # File name
label=[]  # folder name
for subfolder in os.listdir(training_folder):
    sub_folder_path=os.path.join(training_folder,subfolder)
    if os.path.isdir(sub_folder_path):
        for file_name in os.listdir(sub_folder_path):
            path=os.path.join(sub_folder_path,file_name)
            if os.path.isfile(path):
                file_path.append(path)
                label.append(subfolder)

In [7]:
# To create a Dataframe for the training data
data={
    'file_path':file_path,
    'label':label
}
train_df=pd.DataFrame(data)


In [8]:
#Check the Training data
train_df.head()

,file_path,label
0,/kaggle/input/leaf-disease-detection-dataset/d...,Tomato___Late_blight
1,/kaggle/input/leaf-disease-detection-dataset/d...,Tomato___Late_blight
2,/kaggle/input/leaf-disease-detection-dataset/d...,Tomato___Late_blight
3,/kaggle/input/leaf-disease-detection-dataset/d...,Tomato___Late_blight
4,/kaggle/input/leaf-disease-detection-dataset/d...,Tomato___Late_blight


In [9]:
# Shape of the Training Data
train_df.shape

(70295, 2)

In [10]:
#To Get the filename with pathname and folder name for label in Testing folder
file_path=[] # File name
label=[]  # folder name
for subfolder in os.listdir(testing_folder):
    sub_folder_path=os.path.join(testing_folder,subfolder)
    if os.path.isdir(sub_folder_path):
        for file_name in os.listdir(sub_folder_path):
            path=os.path.join(sub_folder_path,file_name)
            if os.path.isfile(path):
                file_path.append(path)
                label.append(subfolder)

In [11]:
# To create a Dataframe for the testing data
data={
    'file_path':file_path,
    'label':label
}
test_df=pd.DataFrame(data)


In [12]:
#Check the Testing data
test_df.head()

,file_path,label
0,/kaggle/input/leaf-disease-detection-dataset/d...,Tomato___Late_blight
1,/kaggle/input/leaf-disease-detection-dataset/d...,Tomato___Late_blight
2,/kaggle/input/leaf-disease-detection-dataset/d...,Tomato___Late_blight
3,/kaggle/input/leaf-disease-detection-dataset/d...,Tomato___Late_blight
4,/kaggle/input/leaf-disease-detection-dataset/d...,Tomato___Late_blight


In [13]:
# Shape of the Testing Data
test_df.shape

(17572, 2)

In [7]:
#To initialize the values
width,height,depth=256,256,3
batch_size=32
class_mode="categorical"
scale=True
rescale=None
if scale:
    rescale=1./255

In [8]:
#Training Data Preprocessing
train_datagen=ImageDataGenerator(
    rescale=rescale,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
                                 )

In [9]:
# Test Data Preprocessing
test_datagen=ImageDataGenerator(
    rescale=rescale,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [10]:
#Training Data
training_data=train_datagen.flow_from_directory(
    training_folder,
    target_size=(width,height),
    batch_size=batch_size,
    class_mode=class_mode,
    subset="training"
    )

Found 56251 images belonging to 38 classes.


In [11]:
#Validation Data
validation_data=train_datagen.flow_from_directory(
    training_folder,
    target_size=(width,height),
    batch_size=batch_size,
    class_mode=class_mode,
    subset="validation"
    )

Found 14044 images belonging to 38 classes.


In [12]:
#Testing Data
testing_data=test_datagen.flow_from_directory(
    testing_folder,
    target_size=(width,height),
    batch_size=batch_size,
    class_mode=class_mode
)

Found 17572 images belonging to 38 classes.


In [13]:
true_labels = testing_data.classes

In [14]:
true_labels

array([ 0,  0,  0, ..., 37, 37, 37], dtype=int32)

In [65]:
#CNN Layers
model=Sequential()
model.add(Conv2D(32, kernel_size=(3,3),input_shape=(width,height,depth), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(96, kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(32, kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(number_of_classes,activation='softmax'))

In [66]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


In [67]:
history=model.fit(
    training_data,
    steps_per_epoch=int(len(training_data)/10),
    epochs=2,
    validation_data=validation_data,
    validation_steps=int(len(validation_data)/10),
    callbacks=[early_stopping]
)

Epoch 1/2


2024-08-03 15:01:56.087583: E external/local_xla/xla/service/gpu/buffer_comparator.cc:153] Difference at 0: 5.22285, expected 4.50617
2024-08-03 15:01:56.087650: E external/local_xla/xla/service/gpu/buffer_comparator.cc:153] Difference at 19: 4.83523, expected 4.11855
2024-08-03 15:01:56.087660: E external/local_xla/xla/service/gpu/buffer_comparator.cc:153] Difference at 20: 6.14347, expected 5.42679
2024-08-03 15:01:56.087668: E external/local_xla/xla/service/gpu/buffer_comparator.cc:153] Difference at 21: 5.72417, expected 5.00749
2024-08-03 15:01:56.087676: E external/local_xla/xla/service/gpu/buffer_comparator.cc:153] Difference at 29: 6.00776, expected 5.29109
2024-08-03 15:01:56.087683: E external/local_xla/xla/service/gpu/buffer_comparator.cc:153] Difference at 30: 5.39609, expected 4.67941
2024-08-03 15:01:56.087691: E external/local_xla/xla/service/gpu/buffer_comparator.cc:153] Difference at 31: 5.14673, expected 4.43005
2024-08-03 15:01:56.087699: E external/local_xla/xla/ser

175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 449ms/step - accuracy: 0.1975 - loss: 3.2081

2024-08-03 15:03:21.813193: E external/local_xla/xla/service/gpu/buffer_comparator.cc:153] Difference at 0: 5.22285, expected 4.50617
2024-08-03 15:03:21.813247: E external/local_xla/xla/service/gpu/buffer_comparator.cc:153] Difference at 19: 4.83523, expected 4.11855
2024-08-03 15:03:21.813256: E external/local_xla/xla/service/gpu/buffer_comparator.cc:153] Difference at 20: 6.14347, expected 5.42679
2024-08-03 15:03:21.813264: E external/local_xla/xla/service/gpu/buffer_comparator.cc:153] Difference at 21: 5.72417, expected 5.00749
2024-08-03 15:03:21.813272: E external/local_xla/xla/service/gpu/buffer_comparator.cc:153] Difference at 29: 6.00776, expected 5.29109
2024-08-03 15:03:21.813280: E external/local_xla/xla/service/gpu/buffer_comparator.cc:153] Difference at 30: 5.39609, expected 4.67941
2024-08-03 15:03:21.813287: E external/local_xla/xla/service/gpu/buffer_comparator.cc:153] Difference at 31: 5.14673, expected 4.43005
2024-08-03 15:03:21.813295: E external/local_xla/xla/ser

175/175 ━━━━━━━━━━━━━━━━━━━━ 114s 576ms/step - accuracy: 0.1980 - loss: 3.2050 - val_accuracy: 0.0414 - val_loss: 10.4955
Epoch 2/2
175/175 ━━━━━━━━━━━━━━━━━━━━ 96s 551ms/step - accuracy: 0.4594 - loss: 1.8639 - val_accuracy: 0.1032 - val_loss: 10.3186


In [ ]:
cnn_loss, cnn_accuracy=model.evaluate(testing_data)
print("Testing Accuracy:",cnn_accuracy)

In [68]:
model.save("cnn.h5")

In [69]:
model=load_model("cnn.h5")

In [ ]:
#y_pred_probs_cnn = model.predict(testing_data)

In [ ]:
#y_pred_cnn = np.argmax(y_pred_probs_cnn, axis=1)

In [ ]:
#report_cnn = classification_report(true_labels, y_pred_cnn)
#print(report_cnn)

In [ ]:
#accuracy_cnn=accuracy_score(true_labels, y_pred_cnn)
#accuracy_cnn

In [ ]:
# Train CNN architectures 


In [82]:
#Xception
base_model = Xception(weights='imagenet', include_top=False, input_shape=(width, height, depth))


In [83]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)  # Adjust units and activation as needed
predictions = Dense(number_of_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)


In [84]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [85]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [86]:
history = model.fit(training_data, steps_per_epoch=int(len(training_data)/10),epochs=20, batch_size=32,validation_data=validation_data, validation_steps=int(len(validation_data)/10), callbacks=[early_stopping])

Epoch 1/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 119s 470ms/step - accuracy: 0.5928 - loss: 1.5436
Epoch 2/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 83s 477ms/step - accuracy: 0.8731 - loss: 0.4325
Epoch 3/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 83s 475ms/step - accuracy: 0.8905 - loss: 0.3557
Epoch 4/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 82s 470ms/step - accuracy: 0.9256 - loss: 0.2667
Epoch 5/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 82s 472ms/step - accuracy: 0.9298 - loss: 0.2275
Epoch 6/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 81s 464ms/step - accuracy: 0.9349 - loss: 0.2210
Epoch 7/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 81s 461ms/step - accuracy: 0.9475 - loss: 0.1920
Epoch 8/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 81s 460ms/step - accuracy: 0.9463 - loss: 0.1921
Epoch 9/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 80s 460ms/step - accuracy: 0.9474 - loss: 0.1828
Epoch 10/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 91s 520ms/step - accuracy: 0.9566 - loss: 0.1443
Epoch 11/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9651 - loss: 0.1371  
Epoch 12/20
175/17

In [87]:
model.save("exception.h5")

In [ ]:
exception_loss, exception_accuracy = model_exception.evaluate(testing_data)
print(f"Testing Accuracy: {exception_accuracy}")

In [27]:
from keras.models import save_model

In [62]:
save_model(model_exception,"exception.h5")

In [80]:
from keras.models import load_model

In [81]:
model=load_model("exception.h5")

In [ ]:
# VGG16

In [ ]:
base_model_vgg16 = VGG16(
    input_shape=(width, height, depth),
    include_top=False,
    weights='imagenet',
    pooling="max",
    classifier_activation="softmax"
)
base_model_vgg16.trainable = False  # Freeze the base model

In [ ]:
model_vgg16 = Sequential([
    base_model_vgg16,
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(number_of_classes, activation='softmax')  # Change the output units based on your number of classes
])

In [ ]:
model_vgg16.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


In [ ]:
history_vgg16=model_vgg16.fit(
    training_data,
    steps_per_epoch=int(len(training_data)/10),
    epochs=20,
    validation_data=validation_data,
    validation_steps=int(len(validation_data)/10),
    callbacks=[early_stopping]
)

In [ ]:
vgg16_loss, vgg16_accuracy = model_vgg16.evaluate(testing_data)
print(f"Testing Accuracy: {vgg16_accuracy}")

In [ ]:
#VGG19

In [ ]:
base_model_vgg19 = VGG19(
    input_shape=(width, height, depth),
    include_top=False,
    weights='imagenet',
    pooling="max",
    classifier_activation="softmax"
)
base_model_vgg19.trainable = False  # Freeze the base model

In [ ]:
model_vgg19 = Sequential([
    base_model_vgg19,
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(number_of_classes, activation='softmax')  # Change the output units based on your number of classes
])

In [ ]:
model_vgg19.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
history_vgg19=model_vgg19.fit(
    training_data,
    steps_per_epoch=int(len(training_data)/10),
    epochs=20,
    validation_data=validation_data,
    validation_steps=int(len(validation_data)/10),
    callbacks=[early_stopping]
)

In [ ]:
vgg19_loss, vgg19_accuracy = model_vgg19.evaluate(testing_data)
print(f"Testing Accuracy: {vgg19_accuracy}")

In [ ]:
#Resnet50

In [ ]:
base_model_resnet50 = ResNet50(
    input_shape=(width, height, depth),
    include_top=False,
    weights='imagenet',
    pooling="max",
    classifier_activation="softmax"
)
base_model_resnet50.trainable = False  # Freeze the base model

In [ ]:
model_resnet50 = Sequential([
    base_model_resnet50,
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(number_of_classes, activation='softmax')  # Change the output units based on your number of classes
])

In [ ]:
model_resnet50.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history_resnet50=model_resnet50.fit(
    training_data,
    steps_per_epoch=int(len(training_data)/10),
    epochs=20,
    validation_data=validation_data,
    validation_steps=int(len(validation_data)/10),
    callbacks=[early_stopping]
)

In [ ]:
resnet50_loss,resnet50_accuracy = model_resnet50.evaluate(testing_data)
print(f"Testing Accuracy: {resnet50_accuracy}")

In [ ]:
#InceptionV3

In [ ]:
base_model_inceptionv3 = InceptionV3(
    input_shape=(width, height, depth),
    include_top=False,
    weights='imagenet',
    pooling="max",
    classifier_activation="softmax"
)
base_model_inceptionv3.trainable = False  # Freeze the base model

In [ ]:
model_inceptionv3 = Sequential([
    base_model_inceptionv3,
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(number_of_classes, activation='softmax')  # Change the output units based on your number of classes
])

In [ ]:
model_inceptionv3.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history_inceptionv3=model_inceptionv3.fit(
    training_data,
    steps_per_epoch=int(len(training_data)/10),
    epochs=20,
    validation_data=validation_data,
    validation_steps=int(len(validation_data)/10),
    callbacks=[early_stopping]
)

In [ ]:
inceptionv3_loss,inceptionv3_accuracy = model_inceptionv3.evaluate(testing_data)
print(f"Testing Accuracy: {inceptionv3_accuracy}")

In [ ]:
def preprocess_image(img_path):
    # Preprocess the image
    #img_path = '/kaggle/input/leaf-disease-detection-dataset/images_for_test/AppleCedarRust2.JPG'
    img = image.load_img(img_path, target_size=(256, 256))  # Adjust target_size as per your model's input size
    # Preprocess the image
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # Normalize the image if required by your model
    return img_array

In [ ]:
def predict_image(img_path):
    # Predict the class
    processed_image=preprocess_image(img_path)
    predictions = model_exception.predict(processed_image)
    # Interpret the output
    predicted_class = np.argmax(predictions, axis=1)
    #print(f'Predicted class: {predicted_class}')
    return predicted_class

In [30]:
class_names=sorted(os.listdir("/kaggle/input/leaf-disease-detection-dataset/dataset/train"))
print(class_names)

['Apple___Apple_scab',
 'Apple___Black_rot',
 'Apple___Cedar_apple_rust',
 'Apple___healthy',
 'Blueberry___healthy',
 'Cherry_(including_sour)___Powdery_mildew',
 'Cherry_(including_sour)___healthy',
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 'Corn_(maize)___Common_rust_',
 'Corn_(maize)___Northern_Leaf_Blight',
 'Corn_(maize)___healthy',
 'Grape___Black_rot',
 'Grape___Esca_(Black_Measles)',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Grape___healthy',
 'Orange___Haunglongbing_(Citrus_greening)',
 'Peach___Bacterial_spot',
 'Peach___healthy',
 'Pepper,_bell___Bacterial_spot',
 'Pepper,_bell___healthy',
 'Potato___Early_blight',
 'Potato___Late_blight',
 'Potato___healthy',
 'Raspberry___healthy',
 'Soybean___healthy',
 'Squash___Powdery_mildew',
 'Strawberry___Leaf_scorch',
 'Strawberry___healthy',
 'Tomato___Bacterial_spot',
 'Tomato___Early_blight',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Septoria_leaf_spot',
 'Tomato___Spider_mites Two-spotted_

In [ ]:
true=[]
pred=[]

In [ ]:
testing_image="/kaggle/input/leaf-disease-detection-dataset/images_for_test"
for images in os.listdir(testing_image):
    img_path=testing_image+'/'+images
    predictions=predict_image(img_path)
    print(images)
    true.append(images)
    result=class_names[predictions[0]]
    print(result)
    pred.append(result)

In [ ]:
for i in range(len(true)):
    print(true[i], pred[i])

In [3]:
import tensorflow as tf
from tensorflow import keras 
print(f"Keras version: {keras.__version__}")
print(f"TensorFlow version: {tf.__version__}")

Keras version: 3.4.1
TensorFlow version: 2.17.0
